In [ ]:
!pip install timm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade sympy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil
from pathlib import Path
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
from torchvision.transforms import TrivialAugmentWide
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# Paths
RAW_PATH = "/content/drive/MyDrive/data_rabies_101"
CROPPED_PATH = "/content/drive/MyDrive/annotaion_101"
OUTPUT_RAW = "/content/drive/MyDrive/rabies_raw_split"
OUTPUT_CROPPED = "/content/drive/MyDrive/rabies_cropped_split"

# Split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

print("=" * 60)
print("RABIES CLASSIFICATION - PROPER DATA SPLITTING")
print("=" * 60)

def get_image_paths(base_path, class_name):
    """Get all image paths for a class"""
    cls_path = os.path.join(base_path, class_name)
    if not os.path.exists(cls_path):
        return []
    return [os.path.join(cls_path, f) for f in os.listdir(cls_path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

def split_and_save(source_path, output_path, dataset_name):
    """Split dataset into train/val/test and save"""

    print(f"\n{dataset_name} Dataset:")
    print("-" * 40)

    # Get images for each class
    positif_images = get_image_paths(source_path, 'positif')
    negatif_images = get_image_paths(source_path, 'negatif')

    print(f"Original counts:")
    print(f"  Positif: {len(positif_images)}")
    print(f"  Negatif: {len(negatif_images)}")

    # Split each class independently to maintain class balance
    splits = {}
    for class_name, images in [('positif', positif_images), ('negatif', negatif_images)]:
        # First split: train vs (val+test)
        train, val_test = train_test_split(images, test_size=(VAL_RATIO + TEST_RATIO),
                                           random_state=42, shuffle=True)

        # Second split: val vs test
        val, test = train_test_split(val_test, test_size=TEST_RATIO/(VAL_RATIO + TEST_RATIO),
                                     random_state=42, shuffle=True)

        splits[class_name] = {'train': train, 'val': val, 'test': test}

    # Create directory structure
    for split_name in ['train', 'val', 'test']:
        for class_name in ['positif', 'negatif']:
            os.makedirs(os.path.join(output_path, split_name, class_name), exist_ok=True)

    # Copy images to respective splits
    split_counts = defaultdict(lambda: defaultdict(int))

    for class_name in ['positif', 'negatif']:
        for split_name in ['train', 'val', 'test']:
            for img_path in splits[class_name][split_name]:
                img_name = os.path.basename(img_path)
                dst = os.path.join(output_path, split_name, class_name, img_name)
                shutil.copy2(img_path, dst)
                split_counts[split_name][class_name] += 1

    # Print split statistics
    print(f"\nSplit distribution:")
    for split_name in ['train', 'val', 'test']:
        pos = split_counts[split_name]['positif']
        neg = split_counts[split_name]['negatif']
        total = pos + neg
        print(f"  {split_name.upper():5s}: Positif={pos:3d}, Negatif={neg:3d}, Total={total:3d}")

    return split_counts

# Split both datasets
raw_splits = split_and_save(RAW_PATH, OUTPUT_RAW, "RAW")
cropped_splits = split_and_save(CROPPED_PATH, OUTPUT_CROPPED, "CROPPED")

# Define augmentation strategies
print("\n" + "=" * 60)
print("AUGMENTATION STRATEGIES (Training Set Only)")
print("=" * 60)

aug_strategies = {
    'TrivialAugmentWide': TrivialAugmentWide(),

    'Geometric_Color': T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.RandomRotation(degrees=20),
        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    ]),

    'Spatial_Blur': T.Compose([
        T.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
    ])
}

def augment_train_set(base_path, aug_name, transform, aug_per_image=3):
    """Augment only the training set"""

    train_path = os.path.join(base_path, 'train')
    aug_train_path = os.path.join(base_path, f'train_{aug_name}')

    stats = defaultdict(int)

    for class_name in ['positif', 'negatif']:
        src_cls = os.path.join(train_path, class_name)
        dst_cls = os.path.join(aug_train_path, class_name)
        os.makedirs(dst_cls, exist_ok=True)

        images = [f for f in os.listdir(src_cls)
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Copy original images
        for img_name in images:
            src = os.path.join(src_cls, img_name)
            dst = os.path.join(dst_cls, img_name)
            shutil.copy2(src, dst)
            stats[f'{class_name}_original'] += 1

        # Generate augmented images
        for img_name in images:
            img_path = os.path.join(src_cls, img_name)
            img = Image.open(img_path).convert('RGB')

            for i in range(aug_per_image):
                aug_img = transform(img)
                base_name = os.path.splitext(img_name)[0]
                aug_name_file = f"aug_{i}_{base_name}.jpg"
                aug_img.save(os.path.join(dst_cls, aug_name_file))
                stats[f'{class_name}_augmented'] += 1

    return stats

# Apply augmentations to training sets only
print("\nAugmenting TRAINING sets (Val/Test remain original):")
print("-" * 60)

for aug_name, transform in aug_strategies.items():
    print(f"\n{aug_name}:")

    # Augment raw training set
    raw_stats = augment_train_set(OUTPUT_RAW, aug_name, transform)
    print(f"  RAW Train - Positif: {raw_stats['positif_original']} orig + "
          f"{raw_stats['positif_augmented']} aug = "
          f"{raw_stats['positif_original'] + raw_stats['positif_augmented']} total")
    print(f"  RAW Train - Negatif: {raw_stats['negatif_original']} orig + "
          f"{raw_stats['negatif_augmented']} aug = "
          f"{raw_stats['negatif_original'] + raw_stats['negatif_augmented']} total")

    # Augment cropped training set
    cropped_stats = augment_train_set(OUTPUT_CROPPED, aug_name, transform)
    print(f"  CROPPED Train - Positif: {cropped_stats['positif_original']} orig + "
          f"{cropped_stats['positif_augmented']} aug = "
          f"{cropped_stats['positif_original'] + cropped_stats['positif_augmented']} total")
    print(f"  CROPPED Train - Negatif: {cropped_stats['negatif_original']} orig + "
          f"{cropped_stats['negatif_augmented']} aug = "
          f"{cropped_stats['negatif_original'] + cropped_stats['negatif_augmented']} total")

print("\n" + "=" * 60)
print("COMPLETE - DATA SPLITTING & AUGMENTATION DONE")
print("=" * 60)
print(f"\nData structure created:")
print(f"  Raw: {OUTPUT_RAW}")
print(f"    ├── train/ (original)")
print(f"    ├── train_TrivialAugmentWide/ (augmented)")
print(f"    ├── train_Geometric_Color/ (augmented)")
print(f"    ├── train_Spatial_Blur/ (augmented)")
print(f"    ├── val/ (original only)")
print(f"    └── test/ (original only)")
print(f"\n  Cropped: {OUTPUT_CROPPED}")
print(f"    └── (same structure)")
print(f"\nVal and Test sets are NEVER augmented - pure original data!")

KeyboardInterrupt: 

In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
import numpy as np

print("=" * 60)
print("STRATIFIED K-FOLD CROSS-VALIDATION SETUP")
print("=" * 60)

# Configuration
N_FOLDS = 5
BATCH_SIZE = 32
NUM_WORKERS = 2
IMG_SIZE = 224
RANDOM_SEED = 42

# Set seeds for reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"\nConfiguration:")
print(f"  Number of Folds: {N_FOLDS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Random Seed: {RANDOM_SEED}")

# Custom Dataset Class
class RabiesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.labels = []

        # Class mapping
        self.class_to_idx = {'negatif': 0, 'positif': 1}

        # Load all images
        for class_name in ['negatif', 'positif']:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                continue

            label = self.class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path = self.samples[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Transforms for training and validation
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# Function to create stratified K-fold splits
def create_stratified_kfold(dataset, n_folds=5, random_seed=42):
    """
    Create stratified K-fold splits maintaining class distribution

    Returns:
        list of (train_indices, val_indices) tuples for each fold
    """
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)

    # Get labels as numpy array
    labels = np.array(dataset.labels)
    indices = np.arange(len(dataset))

    folds = []
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(indices, labels)):
        folds.append((train_idx, val_idx))

        # Print fold statistics
        train_labels = labels[train_idx]
        val_labels = labels[val_idx]

        print(f"\nFold {fold_idx + 1}:")
        print(f"  Train: {len(train_idx)} samples "
              f"(Negatif: {np.sum(train_labels == 0)}, "
              f"Positif: {np.sum(train_labels == 1)})")
        print(f"  Val:   {len(val_idx)} samples "
              f"(Negatif: {np.sum(val_labels == 0)}, "
              f"Positif: {np.sum(val_labels == 1)})")
        print(f"  Train ratio: {np.sum(train_labels == 1)/len(train_labels):.2%} positif")
        print(f"  Val ratio:   {np.sum(val_labels == 1)/len(val_labels):.2%} positif")

    return folds

# Function to get dataloaders for a specific fold
def get_fold_dataloaders(dataset, train_indices, val_indices, batch_size=32, num_workers=2):
    """
    Create train and validation dataloaders for a specific fold
    """
    # Create subsets
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, val_loader

# Test the setup with one dataset configuration
print("\n" + "=" * 60)
print("TESTING K-FOLD SETUP")
print("=" * 60)

# Example: Test with RAW + TrivialAugmentWide training set
test_train_dir = "/content/drive/MyDrive/rabies_raw_split/train_TrivialAugmentWide"
test_dataset = RabiesDataset(test_train_dir, transform=train_transform)

print(f"\nDataset: {test_train_dir}")
print(f"Total samples: {len(test_dataset)}")
print(f"Negatif: {test_dataset.labels.count(0)}")
print(f"Positif: {test_dataset.labels.count(1)}")

# Create folds
folds = create_stratified_kfold(test_dataset, n_folds=N_FOLDS, random_seed=RANDOM_SEED)

print("\n" + "=" * 60)
print("K-FOLD CONFIGURATION COMPLETE")
print("=" * 60)
print(f"\nTo use in training:")
print(f"  1. Load dataset with RabiesDataset(path, transform)")
print(f"  2. Create folds with create_stratified_kfold(dataset, {N_FOLDS})")
print(f"  3. For each fold, get dataloaders with get_fold_dataloaders()")
print(f"\nExample usage:")
print(f"  for fold_idx, (train_idx, val_idx) in enumerate(folds):")
print(f"      train_loader, val_loader = get_fold_dataloaders(dataset, train_idx, val_idx)")
print(f"      # Train model with train_loader, validate with val_loader")

STRATIFIED K-FOLD CROSS-VALIDATION SETUP

Configuration:
  Number of Folds: 5
  Batch Size: 32
  Image Size: 224x224
  Random Seed: 42

TESTING K-FOLD SETUP

Dataset: /content/drive/MyDrive/rabies_raw_split/train_TrivialAugmentWide
Total samples: 432
Negatif: 88
Positif: 344

Fold 1:
  Train: 345 samples (Negatif: 70, Positif: 275)
  Val:   87 samples (Negatif: 18, Positif: 69)
  Train ratio: 79.71% positif
  Val ratio:   79.31% positif

Fold 2:
  Train: 345 samples (Negatif: 70, Positif: 275)
  Val:   87 samples (Negatif: 18, Positif: 69)
  Train ratio: 79.71% positif
  Val ratio:   79.31% positif

Fold 3:
  Train: 346 samples (Negatif: 70, Positif: 276)
  Val:   86 samples (Negatif: 18, Positif: 68)
  Train ratio: 79.77% positif
  Val ratio:   79.07% positif

Fold 4:
  Train: 346 samples (Negatif: 71, Positif: 275)
  Val:   86 samples (Negatif: 17, Positif: 69)
  Train ratio: 79.48% positif
  Val ratio:   80.23% positif

Fold 5:
  Train: 346 samples (Negatif: 71, Positif: 275)
  Val:

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

print("=" * 60)
print("TRAINING CONFIGURATION")
print("=" * 60)

# ============================================================
# HYPERPARAMETERS
# ============================================================
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 32
MAX_EPOCHS = 25
PATIENCE_EARLY_STOP = 10
PATIENCE_LR_SCHEDULER = 5

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"\nHyperparameters:")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Weight Decay (L2): {WEIGHT_DECAY}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Max Epochs: {MAX_EPOCHS}")
print(f"  Early Stop Patience: {PATIENCE_EARLY_STOP} epochs")
print(f"  LR Scheduler Patience: {PATIENCE_LR_SCHEDULER} epochs")
print(f"  Device: {DEVICE}")

# ============================================================
# CLASS WEIGHTS FOR IMBALANCED DATA
# ============================================================
# Based on your data: 344 positif, 88 negatif in augmented training
def calculate_class_weights(n_positif, n_negatif):
    """
    Calculate class weights for imbalanced dataset
    Formula: weight = total_samples / (n_classes * class_count)
    """
    total = n_positif + n_negatif
    weight_positif = total / (2 * n_positif)
    weight_negatif = total / (2 * n_negatif)

    return torch.tensor([weight_negatif, weight_positif], dtype=torch.float32)

# Calculate weights based on augmented training set
class_weights = calculate_class_weights(n_positif=344, n_negatif=88)
print(f"\nClass Weights (for imbalance):")
print(f"  Negatif (0): {class_weights[0]:.4f}")
print(f"  Positif (1): {class_weights[1]:.4f}")
print(f"  → Negatif samples weighted {class_weights[0]/class_weights[1]:.2f}x more")

# ============================================================
# LOSS FUNCTION
# ============================================================
criterion = nn.CrossEntropyLoss(weight=class_weights.to(DEVICE))
print(f"\nLoss Function: CrossEntropyLoss with class weights")

# ============================================================
# OPTIMIZER & SCHEDULER SETUP FUNCTION
# ============================================================
def setup_optimizer_scheduler(model, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY):
    """
    Setup optimizer and learning rate scheduler
    """
    # AdamW optimizer (Adam with weight decay fix)
    optimizer = optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
        betas=(0.9, 0.999)
    )

    # ReduceLROnPlateau: reduce LR when val loss plateaus
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=PATIENCE_LR_SCHEDULER,
        min_lr=1e-7
    )

    return optimizer, scheduler

print(f"\nOptimizer: AdamW")
print(f"  Beta1: 0.9, Beta2: 0.999")
print(f"\nLR Scheduler: ReduceLROnPlateau")
print(f"  Reduces LR by 0.5x when val loss doesn't improve for {PATIENCE_LR_SCHEDULER} epochs")
print(f"  Minimum LR: 1e-7")

# ============================================================
# EARLY STOPPING CLASS
# ============================================================
class EarlyStopping:
    """Early stops training if validation loss doesn't improve"""

    def __init__(self, patience=15, min_delta=0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# ============================================================
# MODEL CHECKPOINT CLASS
# ============================================================
class ModelCheckpoint:
    """Save best model based on validation metric"""

    def __init__(self, save_path, metric='loss', mode='min', verbose=True):
        self.save_path = save_path
        self.metric = metric
        self.mode = mode
        self.verbose = verbose
        self.best_score = None

    def __call__(self, model, val_metric, epoch, fold=None):
        score = val_metric

        # Check if this is the best score
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, epoch, fold)
        elif (self.mode == 'min' and score < self.best_score) or \
             (self.mode == 'max' and score > self.best_score):
            self.best_score = score
            self.save_checkpoint(model, epoch, fold)

    def save_checkpoint(self, model, epoch, fold):
        if self.verbose:
            fold_str = f"Fold {fold}, " if fold is not None else ""
            print(f'Saving checkpoint: {fold_str}Epoch {epoch}, '
                  f'{self.metric}={self.best_score:.4f}')
        torch.save(model.state_dict(), self.save_path)

print(f"\n" + "=" * 60)
print("EARLY STOPPING & CHECKPOINTING")
print("=" * 60)
print(f"\nEarly Stopping:")
print(f"  Monitors: Validation Loss")
print(f"  Patience: {PATIENCE_EARLY_STOP} epochs")
print(f"  Action: Stop training if no improvement")

print(f"\nModel Checkpointing:")
print(f"  Monitors: Validation Loss")
print(f"  Saves: Best model weights only")
print(f"  Location: Will be specified per model/fold")

# ============================================================
# TRAINING CONFIGURATION SUMMARY
# ============================================================
print(f"\n" + "=" * 60)
print("CONFIGURATION SUMMARY")
print("=" * 60)

config_summary = {
    'learning_rate': LEARNING_RATE,
    'optimizer': 'AdamW',
    'weight_decay': WEIGHT_DECAY,
    'batch_size': BATCH_SIZE,
    'max_epochs': MAX_EPOCHS,
    'early_stopping_patience': PATIENCE_EARLY_STOP,
    'lr_scheduler': 'ReduceLROnPlateau',
    'lr_scheduler_patience': PATIENCE_LR_SCHEDULER,
    'loss_function': 'CrossEntropyLoss',
    'class_weights': f'[{class_weights[0]:.4f}, {class_weights[1]:.4f}]',
    'device': str(DEVICE)
}

for key, value in config_summary.items():
    print(f"  {key}: {value}")

print(f"\n✓ Training configuration complete and ready!")
print(f"✓ Optimized for imbalanced dataset (79% positif, 21% negatif)")
print(f"✓ Prevents overfitting with early stopping & L2 regularization")

TRAINING CONFIGURATION

Hyperparameters:
  Learning Rate: 0.0001
  Weight Decay (L2): 0.0001
  Batch Size: 32
  Max Epochs: 25
  Early Stop Patience: 10 epochs
  LR Scheduler Patience: 5 epochs
  Device: cuda

Class Weights (for imbalance):
  Negatif (0): 2.4545
  Positif (1): 0.6279
  → Negatif samples weighted 3.91x more

Loss Function: CrossEntropyLoss with class weights

Optimizer: AdamW
  Beta1: 0.9, Beta2: 0.999

LR Scheduler: ReduceLROnPlateau
  Reduces LR by 0.5x when val loss doesn't improve for 5 epochs
  Minimum LR: 1e-7

EARLY STOPPING & CHECKPOINTING

Early Stopping:
  Monitors: Validation Loss
  Patience: 10 epochs
  Action: Stop training if no improvement

Model Checkpointing:
  Monitors: Validation Loss
  Saves: Best model weights only
  Location: Will be specified per model/fold

CONFIGURATION SUMMARY
  learning_rate: 0.0001
  optimizer: AdamW
  weight_decay: 0.0001
  batch_size: 32
  max_epochs: 25
  early_stopping_patience: 10
  lr_scheduler: ReduceLROnPlateau
  lr_s

In [5]:
import json
import os
from datetime import datetime
import torch

print("=" * 60)
print("HYPERPARAMETERS & METADATA SAVER")
print("=" * 60)

# Define RESULTS_DIR here to ensure it's available
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"

# ============================================================
# COMPLETE CONFIGURATION FOR RESEARCH PAPER
# ============================================================

RESEARCH_CONFIG = {
    # Dataset Information
    "dataset": {
        "name": "Rabies Classification Dataset",
        "total_samples": {
            "raw": {"positif": 123, "negatif": 32},
            "cropped": {"positif": 123, "negatif": 32}
        },
        "train_val_test_split": {
            "train": 0.70,
            "validation": 0.15,
            "test": 0.15
        },
        "after_split": {
            "train": {"positif": 86, "negatif": 22},
            "validation": {"positif": 18, "negatif": 5},
            "test": {"positif": 19, "negatif": 5}
        },
        "augmentations_per_image": 3,
        "augmented_train_size": {
            "positif": 344,  # 86 * 4 (original + 3 augmented)
            "negatif": 88    # 22 * 4
        },
        "class_imbalance_ratio": "79.6% positif : 20.4% negatif"
    },

    # Training Configuration
    "training": {
        "cross_validation": {
            "method": "Stratified K-Fold",
            "n_folds": 5,
            "random_seed": 42
        },
        "hyperparameters": {
            "learning_rate": 1e-4,
            "optimizer": "AdamW",
            "optimizer_params": {
                "betas": [0.9, 0.999],
                "weight_decay": 1e-4
            },
            "batch_size": 32,
            "max_epochs": 40,
            "image_size": 224
        },
        "regularization": {
            "weight_decay_l2": 1e-4,
            "early_stopping_patience": 15,
            "dropout": "None (using pretrained models)"
        },
        "loss_function": {
            "type": "CrossEntropyLoss",
            "class_weights": {
                "negatif": 2.4545,
                "positif": 0.6279
            },
            "reason": "Handle class imbalance"
        },
        "learning_rate_scheduler": {
            "type": "ReduceLROnPlateau",
            "mode": "min",
            "factor": 0.5,
            "patience": 5,
            "min_lr": 1e-7
        }
    },

    # Models
    "models": {
        "EfficientNet-B0": {
            "architecture": "EfficientNet-B0",
            "pretrained": "ImageNet",
            "parameters": "~5.3M",
            "input_size": "224x224"
        },
        "EfficientNet-B2": {
            "architecture": "EfficientNet-B2",
            "pretrained": "ImageNet",
            "parameters": "~9.2M",
            "input_size": "224x224"
        },
        "VGG16": {
            "architecture": "VGG16",
            "pretrained": "ImageNet",
            "parameters": "~138M",
            "input_size": "224x224"
        },
        "ViT-B-16": {
            "architecture": "Vision Transformer Base",
            "pretrained": "ImageNet",
            "parameters": "~86M",
            "patch_size": 16,
            "input_size": "224x224"
        }
    },

    # Augmentation Strategies
    "augmentation_strategies": {
        "TrivialAugmentWide": {
            "description": "Automated augmentation policy",
            "operations": "Random selection from 14 operations",
            "library": "torchvision.transforms"
        },
        "Geometric_Color": {
            "description": "Geometric and color transformations",
            "operations": [
                "RandomHorizontalFlip (p=0.5)",
                "RandomVerticalFlip (p=0.5)",
                "RandomRotation (degrees=20)",
                "ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)"
            ]
        },
        "Spatial_Blur": {
            "description": "Spatial transformations and blur",
            "operations": [
                "RandomAffine (translate=0.1, scale=0.9-1.1)",
                "GaussianBlur (kernel_size=3, sigma=0.1-2.0)"
            ]
        }
    },

    # Data Processing
    "preprocessing": {
        "normalization": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
            "note": "ImageNet normalization"
        },
        "resize": "224x224",
        "color_space": "RGB"
    },

    # Hardware & Software
    "environment": {
        "framework": "PyTorch",
        "device": str(torch.device('cuda' if torch.cuda.is_available() else 'cpu')),
        "cuda_available": torch.cuda.is_available(),
        "num_workers": 2
    }
}

# ============================================================
# SAVE CONFIGURATION
# ============================================================

def save_research_config(save_dir):
    """Save complete research configuration"""
    os.makedirs(save_dir, exist_ok=True)

    # Add timestamp
    RESEARCH_CONFIG["metadata"] = {
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "purpose": "Rabies Classification Research Paper",
        "comparison": "Raw vs Cropped microscopic images"
    }

    # Save as JSON
    config_path = os.path.join(save_dir, "research_configuration.json")
    with open(config_path, 'w') as f:
        json.dump(RESEARCH_CONFIG, f, indent=2)

    print(f"✓ Research configuration saved: {config_path}")

    # Also save a formatted text version for easy reading
    txt_path = os.path.join(save_dir, "research_configuration.txt")
    with open(txt_path, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("RABIES CLASSIFICATION - RESEARCH CONFIGURATION\n")
        f.write("=" * 70 + "\n\n")

        f.write("DATASET INFORMATION\n")
        f.write("-" * 70 + "\n")
        f.write(f"Total Samples: Positif={RESEARCH_CONFIG['dataset']['total_samples']['raw']['positif']}, ")
        f.write(f"Negatif={RESEARCH_CONFIG['dataset']['total_samples']['raw']['negatif']}\n")
        f.write(f"Train/Val/Test Split: {RESEARCH_CONFIG['dataset']['train_val_test_split']}\n")
        f.write(f"Class Imbalance: {RESEARCH_CONFIG['dataset']['class_imbalance_ratio']}\n")
        f.write(f"Augmentations per image: {RESEARCH_CONFIG['dataset']['augmentations_per_image']}\n\n")

        f.write("TRAINING CONFIGURATION\n")
        f.write("-" * 70 + "\n")
        f.write(f"Cross-Validation: {RESEARCH_CONFIG['training']['cross_validation']['method']} ")
        f.write(f"(K={RESEARCH_CONFIG['training']['cross_validation']['n_folds']})\n")
        f.write(f"Learning Rate: {RESEARCH_CONFIG['training']['hyperparameters']['learning_rate']}\n")
        f.write(f"Optimizer: {RESEARCH_CONFIG['training']['hyperparameters']['optimizer']}\n")
        f.write(f"Batch Size: {RESEARCH_CONFIG['training']['hyperparameters']['batch_size']}\n")
        f.write(f"Max Epochs: {RESEARCH_CONFIG['training']['hyperparameters']['max_epochs']}\n")
        f.write(f"Early Stopping Patience: {RESEARCH_CONFIG['training']['regularization']['early_stopping_patience']}\n\n")

        f.write("MODELS\n")
        f.write("-" * 70 + "\n")
        for model_name, model_info in RESEARCH_CONFIG['models'].items():
            f.write(f"- {model_name}: {model_info['parameters']} parameters\n")
        f.write("\n")

        f.write("AUGMENTATION STRATEGIES\n")
        f.write("-" * 70 + "\n")
        for aug_name, aug_info in RESEARCH_CONFIG['augmentation_strategies'].items():
            f.write(f"- {aug_name}: {aug_info['description']}\n")

    print(f"✓ Text configuration saved: {txt_path}")

    return config_path, txt_path

# Save the configuration
config_path, txt_path = save_research_config(RESULTS_DIR)

print("\n" + "=" * 60)
print("CONFIGURATION SUMMARY FOR PAPER")
print("=" * 60)
print(f"\n📊 Total Training Configurations:")
print(f"   4 Models × 3 Augmentations × 2 Data Types × 5 Folds = 120 runs")
print(f"\n📁 All results will be saved to: {RESULTS_DIR}")
print(f"   - Model weights (.pth files)")
print(f"   - Training history (JSON)")
print(f"   - Research configuration (JSON + TXT)")
print(f"   - Evaluation metrics (will be generated)")
print(f"\n✓ Ready for research paper documentation!")


HYPERPARAMETERS & METADATA SAVER
✓ Research configuration saved: /content/drive/MyDrive/rabies_results/research_configuration.json
✓ Text configuration saved: /content/drive/MyDrive/rabies_results/research_configuration.txt

CONFIGURATION SUMMARY FOR PAPER

📊 Total Training Configurations:
   4 Models × 3 Augmentations × 2 Data Types × 5 Folds = 120 runs

📁 All results will be saved to: /content/drive/MyDrive/rabies_results
   - Model weights (.pth files)
   - Training history (JSON)
   - Research configuration (JSON + TXT)
   - Evaluation metrics (will be generated)

✓ Ready for research paper documentation!


In [6]:
import torch
import torch.nn as nn
from torchvision import models
import timm
from tqdm import tqdm
import time
import os

print("=" * 60)
print("MODEL DEFINITIONS & TRAINING LOOP")
print("=" * 60)

# ============================================================
# MODEL LOADING FUNCTIONS
# ============================================================

def load_efficientnet_b0(num_classes=2, pretrained=True):
    """Load EfficientNet-B0 with custom classifier"""
    weights = models.EfficientNet_B0_Weights.DEFAULT if pretrained else None
    model = models.efficientnet_b0(weights=weights)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model

def load_efficientnet_b2(num_classes=2, pretrained=True):
    """Load EfficientNet-B2 with custom classifier"""
    weights = models.EfficientNet_B2_Weights.DEFAULT if pretrained else None
    model = models.efficientnet_b2(weights=weights)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model

def load_vgg16(num_classes=2, pretrained=True):
    """Load VGG16 with custom classifier"""
    weights = models.VGG16_Weights.DEFAULT if pretrained else None
    model = models.vgg16(weights=weights)
    in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features, num_classes)
    return model

def load_vit_b16(num_classes=2, pretrained=True):
    """Load Vision Transformer B-16 with custom head"""
    model = timm.create_model('vit_base_patch16_224', pretrained=pretrained)
    in_features = model.head.in_features
    model.head = nn.Linear(in_features, num_classes)
    return model

print("\nAvailable Models:")
print("  1. EfficientNet-B0")
print("  2. EfficientNet-B2")
print("  3. VGG16")
print("  4. ViT-B-16")

# ============================================================
# TRAINING FUNCTION
# ============================================================

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

# ============================================================
# VALIDATION FUNCTION
# ============================================================

def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc, all_preds, all_labels

# ============================================================
# MAIN TRAINING LOOP
# ============================================================

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                device, max_epochs, early_stopping, checkpoint, fold_num,
                model_name, aug_name, data_type):
    """
    Complete training loop with early stopping and checkpointing

    Returns:
        history: Dictionary with training history
    """

    print(f"\n{'='*60}")
    print(f"Training: {model_name} | {data_type} | {aug_name} | Fold {fold_num}")
    print(f"{'='*60}")

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }

    best_val_acc = 0.0
    start_time = time.time()

    for epoch in range(max_epochs):
        epoch_start = time.time()

        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)

        # Train
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion,
                                                  optimizer, device)

        # Validate
        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)

        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Update learning rate
        scheduler.step(val_loss)

        # Checkpointing
        checkpoint(model, val_loss, epoch, fold_num)

        # Track best accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc

        # Print epoch results
        epoch_time = time.time() - epoch_start
        print(f"Epoch [{epoch+1}/{max_epochs}] ({epoch_time:.1f}s) | "
              f"LR: {current_lr:.2e} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        # Early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            break

    total_time = time.time() - start_time
    print(f"\nTraining completed in {total_time/60:.2f} minutes")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")

    return history

# ============================================================
# MODEL FACTORY
# ============================================================

MODEL_FACTORY = {
    'EfficientNet-B0': load_efficientnet_b0,
    'EfficientNet-B2': load_efficientnet_b2,
    'VGG16': load_vgg16,
    'ViT-B-16': load_vit_b16
}

def get_model(model_name, num_classes=2, pretrained=True):
    """Get model by name"""
    if model_name not in MODEL_FACTORY:
        raise ValueError(f"Model {model_name} not supported")
    return MODEL_FACTORY[model_name](num_classes, pretrained)

print(f"\n{'='*60}")
print("TRAINING FUNCTIONS READY")
print(f"{'='*60}")
print("\nUsage:")
print("  1. model = get_model('EfficientNet-B0')")
print("  2. optimizer, scheduler = setup_optimizer_scheduler(model)")
print("  3. history = train_model(model, train_loader, val_loader, ...)")
print("\nAll models will be saved to Drive during training")

MODEL DEFINITIONS & TRAINING LOOP

Available Models:
  1. EfficientNet-B0
  2. EfficientNet-B2
  3. VGG16
  4. ViT-B-16

TRAINING FUNCTIONS READY

Usage:
  1. model = get_model('EfficientNet-B0')
  2. optimizer, scheduler = setup_optimizer_scheduler(model)
  3. history = train_model(model, train_loader, val_loader, ...)

All models will be saved to Drive during training


In [7]:
import os
import json
import pickle
import torch
from torch.utils.data import Subset, DataLoader

print("=" * 60)
print("COMPLETE TRAINING ORCHESTRATOR (3-FOLD + PROGRESSIVE CLEANUP)")
print("=" * 60)

# ============================================================
# CONFIGURATION
# ============================================================

# Paths
BASE_RAW = "/content/drive/MyDrive/rabies_raw_split"
BASE_CROPPED = "/content/drive/MyDrive/rabies_cropped_split"
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"

# Create results directory
os.makedirs(RESULTS_DIR, exist_ok=True)

# Models to train
MODELS = ['EfficientNet-B0', 'EfficientNet-B2', 'VGG16', 'ViT-B-16']

# Augmentation strategies (folder names)
AUGMENTATIONS = ['TrivialAugmentWide', 'Geometric_Color', 'Spatial_Blur']

# Data types
DATA_TYPES = ['raw', 'cropped']

print(f"\nConfiguration:")
print(f"  Models: {len(MODELS)} ({', '.join(MODELS)})")
print(f"  Augmentations: {len(AUGMENTATIONS)} ({', '.join(AUGMENTATIONS)})")
print(f"  Data Types: {len(DATA_TYPES)} ({', '.join(DATA_TYPES)})")
print(f"  K-Folds: {N_FOLDS}")
print(f"  Total training runs: {len(MODELS) * len(AUGMENTATIONS) * len(DATA_TYPES) * N_FOLDS}")
print(f"\n⚡ Progressive cleanup enabled - saves storage space!")

# ============================================================
# RESULTS STORAGE
# ============================================================

def save_results(results, filename):
    """Save results to Drive"""
    filepath = os.path.join(RESULTS_DIR, filename)
    results_dict = {}
    for k1, v1 in results.items():
        results_dict[k1] = {}
        for k2, v2 in v1.items():
            results_dict[k1][k2] = {}
            for k3, v3 in v2.items():
                results_dict[k1][k2][k3] = v3

    with open(filepath, 'wb') as f:
        pickle.dump(results_dict, f)
    print(f"Results saved: {filepath}")

def save_history_json(history, filepath):
    """Save training history as JSON"""
    with open(filepath, 'w') as f:
        json.dump(history, f, indent=2)

def cleanup_fold_checkpoints(model_dir, n_folds):
    """Delete all fold checkpoints to save space"""
    deleted_size = 0
    for fold_idx in range(n_folds):
        checkpoint_path = os.path.join(model_dir, f'fold_{fold_idx+1}_best.pth')
        if os.path.exists(checkpoint_path):
            size = os.path.getsize(checkpoint_path) / (1024 * 1024)
            os.remove(checkpoint_path)
            deleted_size += size
    if deleted_size > 0:
        print(f"  💾 Cleaned up {deleted_size:.1f} MB of checkpoints")
    return deleted_size

# ============================================================
# MAIN ORCHESTRATOR
# ============================================================

def run_complete_training():
    """
    Run complete training pipeline with progressive cleanup
    """

    all_results = {}
    total_space_saved = 0

    complete_results_path = os.path.join(RESULTS_DIR, 'complete_results.pkl')
    if os.path.exists(complete_results_path):
        print("📂 Found existing results - loading...")
        with open(complete_results_path, 'rb') as f:
            all_results = pickle.load(f)
        print("✓ Existing results loaded - will skip completed runs")

    for data_type in DATA_TYPES:
        print(f"\n{'#'*60}")
        print(f"# DATA TYPE: {data_type.upper()}")
        print(f"{'#'*60}")

        base_path = BASE_RAW if data_type == 'raw' else BASE_CROPPED

        for aug_name in AUGMENTATIONS:
            print(f"\n{'='*60}")
            print(f"AUGMENTATION: {aug_name}")
            print(f"{'='*60}")

            train_dir = os.path.join(base_path, f'train_{aug_name}')
            dataset = RabiesDataset(train_dir, transform=train_transform)

            folds = create_stratified_kfold(dataset, n_folds=N_FOLDS)

            for model_name in MODELS:
                print(f"\n{'-'*60}")
                print(f"MODEL: {model_name}")
                print(f"{'-'*60}")

                model_dir = os.path.join(RESULTS_DIR, 'checkpoints',
                                        data_type, aug_name, model_name)
                os.makedirs(model_dir, exist_ok=True)

                fold_results = []

                for fold_idx, (train_idx, val_idx) in enumerate(folds):
                    checkpoint_path = os.path.join(model_dir, f'fold_{fold_idx+1}_best.pth')
                    history_path = os.path.join(model_dir, f'fold_{fold_idx+1}_history.json')

                    if os.path.exists(history_path):
                        print(f"\n>>> Fold {fold_idx + 1}/{N_FOLDS} - ALREADY TRAINED ✓ (Skipping)")
                        with open(history_path, 'r') as f:
                            history = json.load(f)
                        fold_results.append(history)
                        continue

                    print(f"\n>>> Fold {fold_idx + 1}/{N_FOLDS} - TRAINING NOW...")

                    batch_size_to_use = 16 if model_name == 'ViT-B-16' else BATCH_SIZE
                    num_workers = 4 if torch.cuda.is_available() else 2

                    train_loader = DataLoader(
                        Subset(dataset, train_idx),
                        batch_size=batch_size_to_use,
                        shuffle=True,
                        num_workers=num_workers,
                        pin_memory=True,
                        persistent_workers=True if num_workers > 0 else False,
                        prefetch_factor=2
                    )

                    val_loader = DataLoader(
                        Subset(dataset, val_idx),
                        batch_size=batch_size_to_use,
                        shuffle=False,
                        num_workers=num_workers,
                        pin_memory=True,
                        persistent_workers=True if num_workers > 0 else False,
                        prefetch_factor=2
                    )

                    model = get_model(model_name).to(DEVICE)
                    optimizer, scheduler = setup_optimizer_scheduler(model)
                    early_stopping = EarlyStopping(patience=PATIENCE_EARLY_STOP)
                    checkpoint = ModelCheckpoint(checkpoint_path, mode='min')

                    history = train_model(
                        model=model,
                        train_loader=train_loader,
                        val_loader=val_loader,
                        criterion=criterion,
                        optimizer=optimizer,
                        scheduler=scheduler,
                        device=DEVICE,
                        max_epochs=MAX_EPOCHS,
                        early_stopping=early_stopping,
                        checkpoint=checkpoint,
                        fold_num=fold_idx+1,
                        model_name=model_name,
                        aug_name=aug_name,
                        data_type=data_type
                    )

                    save_history_json(history, history_path)
                    fold_results.append(history)

                    del model, optimizer, scheduler
                    torch.cuda.empty_cache()
                    import gc
                    gc.collect()

                print(f"\n🧹 Model {model_name} complete - cleaning up checkpoints...")
                space_saved = cleanup_fold_checkpoints(model_dir, N_FOLDS)
                total_space_saved += space_saved

                if data_type not in all_results:
                    all_results[data_type] = {}
                if aug_name not in all_results[data_type]:
                    all_results[data_type][aug_name] = {}
                all_results[data_type][aug_name][model_name] = fold_results

                save_results(all_results, f'results_{data_type}_{aug_name}_{model_name}.pkl')

    save_results(all_results, 'complete_results.pkl')

    print(f"\n{'#'*60}")
    print("TRAINING COMPLETE!")
    print(f"{'#'*60}")
    print(f"\nAll results saved to: {RESULTS_DIR}")
    print(f"💾 Total space saved by cleanup: {total_space_saved/1024:.2f} GB")

    return all_results

# ============================================================
# QUICK START FUNCTION
# ============================================================

def train_single_config(model_name, data_type, aug_name):
    """Train a single configuration for testing"""

    print(f"\nTraining single configuration:")
    print(f"  Model: {model_name}")
    print(f"  Data: {data_type}")
    print(f"  Augmentation: {aug_name}")

    base_path = BASE_RAW if data_type == 'raw' else BASE_CROPPED
    train_dir = os.path.join(base_path, f'train_{aug_name}')

    dataset = RabiesDataset(train_dir, transform=train_transform)
    folds = create_stratified_kfold(dataset, n_folds=N_FOLDS)

    train_idx, val_idx = folds[0]
    train_loader, val_loader = get_fold_dataloaders(dataset, train_idx, val_idx, BATCH_SIZE)

    model = get_model(model_name).to(DEVICE)
    optimizer, scheduler = setup_optimizer_scheduler(model)
    early_stopping = EarlyStopping(patience=PATIENCE_EARLY_STOP)

    checkpoint_path = os.path.join(RESULTS_DIR, f'test_{model_name}.pth')
    checkpoint = ModelCheckpoint(checkpoint_path, mode='min')

    history = train_model(
        model, train_loader, val_loader, criterion, optimizer, scheduler,
        DEVICE, MAX_EPOCHS, early_stopping, checkpoint, 1,
        model_name, aug_name, data_type
    )

    return history

print(f"\n{'='*60}")
print("ORCHESTRATOR READY")
print(f"{'='*60}")
print("\nTo start complete training:")
print("  results = run_complete_training()")
print("\nTo test single configuration:")
print("  history = train_single_config('EfficientNet-B0', 'raw', 'TrivialAugmentWide')")
print("\n📊 Training Summary:")
print(f"  - 72 total runs (4 models × 3 augmentations × 2 data types × 3 folds)")
print(f"  - Estimated time: 10-12 hours")
print(f"  - Max storage at any time: ~5 GB (with progressive cleanup)")

COMPLETE TRAINING ORCHESTRATOR (3-FOLD + PROGRESSIVE CLEANUP)

Configuration:
  Models: 4 (EfficientNet-B0, EfficientNet-B2, VGG16, ViT-B-16)
  Augmentations: 3 (TrivialAugmentWide, Geometric_Color, Spatial_Blur)
  Data Types: 2 (raw, cropped)
  K-Folds: 5
  Total training runs: 120

⚡ Progressive cleanup enabled - saves storage space!

ORCHESTRATOR READY

To start complete training:
  results = run_complete_training()

To test single configuration:
  history = train_single_config('EfficientNet-B0', 'raw', 'TrivialAugmentWide')

📊 Training Summary:
  - 72 total runs (4 models × 3 augmentations × 2 data types × 3 folds)
  - Estimated time: 10-12 hours
  - Max storage at any time: ~5 GB (with progressive cleanup)


In [ ]:
results = run_complete_training()


############################################################
# DATA TYPE: RAW
############################################################

AUGMENTATION: TrivialAugmentWide

Fold 1:
  Train: 345 samples (Negatif: 70, Positif: 275)
  Val:   87 samples (Negatif: 18, Positif: 69)
  Train ratio: 79.71% positif
  Val ratio:   79.31% positif

Fold 2:
  Train: 345 samples (Negatif: 70, Positif: 275)
  Val:   87 samples (Negatif: 18, Positif: 69)
  Train ratio: 79.71% positif
  Val ratio:   79.31% positif

Fold 3:
  Train: 346 samples (Negatif: 70, Positif: 276)
  Val:   86 samples (Negatif: 18, Positif: 68)
  Train ratio: 79.77% positif
  Val ratio:   79.07% positif

Fold 4:
  Train: 346 samples (Negatif: 71, Positif: 275)
  Val:   86 samples (Negatif: 17, Positif: 69)
  Train ratio: 79.48% positif
  Val ratio:   80.23% positif

Fold 5:
  Train: 346 samples (Negatif: 71, Positif: 275)
  Val:   86 samples (Negatif: 17, Positif: 69)
  Train ratio: 79.48% positif
  Val ratio:   80.23% positif



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:02<00:00, 186MB/s]



Training: VGG16 | cropped | TrivialAugmentWide | Fold 3


...

Epoch [7/25] (12.9s) | LR: 1.00e-04 | Train Loss: 0.7055, Acc: 48.84% | Val Loss: 0.5619, Acc: 43.02%
EarlyStopping counter: 6/10


Epoch [8/25] (12.9s) | LR: 5.00e-05 | Train Loss: 0.6017, Acc: 67.05% | Val Loss: 0.5654, Acc: 44.19%
EarlyStopping counter: 7/10


Epoch [9/25] (13.1s) | LR: 5.00e-05 | Train Loss: 0.5990, Acc: 46.24% | Val Loss: 0.5918, Acc: 43.02%
EarlyStopping counter: 8/10


Epoch [10/25] (12.7s) | LR: 5.00e-05 | Train Loss: 0.5970, Acc: 48.27% | Val Loss: 0.6096, Acc: 43.02%
EarlyStopping counter: 9/10


Epoch [11/25] (12.9s) | LR: 5.00e-05 | Train Loss: 0.5814, Acc: 56.65% | Val Loss: 0.7429, Acc: 40.70%
EarlyStopping counter: 10/10

Early stopping triggered at epoch 11

Training completed in 2.39 minutes
Best validation accuracy: 80.23%

🧹 Model ViT-B-16 complete - cleaning up checkpoints...
  💾 Cleaned up 1636.8 MB of checkpoints
Results saved: /content/drive/MyDrive/rabies_results/results_cropped_Spatial_Blur_ViT-B-16.pkl
Results saved: /content/drive/MyDrive/rabies_results/complete_results.pkl

############################################################
TRAINING COMPLETE!
############################################################

All results saved to: /content/drive/MyDrive/rabies_results
💾 Total space saved by cleanup: 12.24 GB


In [8]:
import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict

print("=" * 60)
print("STEP 6: RESULTS ANALYSIS & METRICS CALCULATION")
print("=" * 60)

# Paths
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"
CHECKPOINT_DIR = os.path.join(RESULTS_DIR, "checkpoints")
FIGURES_DIR = os.path.join(RESULTS_DIR, "figures")
TABLES_DIR = os.path.join(RESULTS_DIR, "tables")

# Create output directories
os.makedirs(FIGURES_DIR, exist_ok=True)
os.makedirs(TABLES_DIR, exist_ok=True)

# Configuration
MODELS = ['EfficientNet-B0', 'EfficientNet-B2', 'VGG16', 'ViT-B-16']
AUGMENTATIONS = ['TrivialAugmentWide', 'Geometric_Color', 'Spatial_Blur']
DATA_TYPES = ['raw', 'cropped']
N_FOLDS = 3

print(f"\nOutput directories created:")
print(f"  Figures: {FIGURES_DIR}")
print(f"  Tables: {TABLES_DIR}")

# ============================================================
# LOAD ALL TRAINING HISTORIES
# ============================================================

def load_all_histories():
    """Load all training histories from JSON files"""

    all_histories = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    print("\n" + "=" * 60)
    print("LOADING TRAINING HISTORIES")
    print("=" * 60)

    total_loaded = 0
    missing = []

    for data_type in DATA_TYPES:
        for aug_name in AUGMENTATIONS:
            for model_name in MODELS:
                model_dir = os.path.join(CHECKPOINT_DIR, data_type, aug_name, model_name)

                if not os.path.exists(model_dir):
                    missing.append(f"{data_type}/{aug_name}/{model_name}")
                    continue

                fold_histories = []
                for fold_idx in range(1, N_FOLDS + 1):
                    history_path = os.path.join(model_dir, f'fold_{fold_idx}_history.json')

                    if os.path.exists(history_path):
                        with open(history_path, 'r') as f:
                            history = json.load(f)
                            fold_histories.append(history)
                            total_loaded += 1
                    else:
                        missing.append(f"{data_type}/{aug_name}/{model_name}/fold_{fold_idx}")

                if fold_histories:
                    all_histories[data_type][aug_name][model_name] = fold_histories

    print(f"\n✓ Loaded {total_loaded} training histories")

    if missing:
        print(f"\n⚠️  Missing {len(missing)} configurations:")
        for m in missing[:10]:
            print(f"   - {m}")
        if len(missing) > 10:
            print(f"   ... and {len(missing) - 10} more")

    return all_histories

# Load histories
histories = load_all_histories()

# ============================================================
# CALCULATE METRICS
# ============================================================

def calculate_metrics_from_history(history):
    """Calculate best metrics from a single fold history"""

    best_val_acc = max(history['val_acc'])
    best_val_loss = min(history['val_loss'])

    # Find epoch with best validation accuracy
    best_epoch = history['val_acc'].index(best_val_acc)

    # Get corresponding training metrics
    train_acc_at_best = history['train_acc'][best_epoch]
    train_loss_at_best = history['train_loss'][best_epoch]

    return {
        'best_val_acc': best_val_acc,
        'best_val_loss': best_val_loss,
        'best_train_acc': train_acc_at_best,
        'best_train_loss': train_loss_at_best,
        'best_epoch': best_epoch + 1,
        'total_epochs': len(history['val_acc'])
    }

def calculate_aggregate_metrics(fold_histories):
    """Calculate mean ± std across folds"""

    fold_metrics = [calculate_metrics_from_history(h) for h in fold_histories]

    val_accs = [m['best_val_acc'] for m in fold_metrics]
    val_losses = [m['best_val_loss'] for m in fold_metrics]
    train_accs = [m['best_train_acc'] for m in fold_metrics]
    epochs = [m['total_epochs'] for m in fold_metrics]

    return {
        'val_acc_mean': np.mean(val_accs),
        'val_acc_std': np.std(val_accs),
        'val_loss_mean': np.mean(val_losses),
        'val_loss_std': np.std(val_losses),
        'train_acc_mean': np.mean(train_accs),
        'train_acc_std': np.std(train_accs),
        'avg_epochs': np.mean(epochs),
        'fold_accuracies': val_accs
    }

# Calculate metrics for all configurations
print("\n" + "=" * 60)
print("CALCULATING METRICS")
print("=" * 60)

all_metrics = defaultdict(lambda: defaultdict(dict))

for data_type in DATA_TYPES:
    for aug_name in AUGMENTATIONS:
        for model_name in MODELS:
            if model_name in histories[data_type][aug_name]:
                fold_histories = histories[data_type][aug_name][model_name]
                metrics = calculate_aggregate_metrics(fold_histories)
                all_metrics[data_type][aug_name][model_name] = metrics

                print(f"{model_name:20s} | {data_type:8s} | {aug_name:20s} | "
                      f"Val Acc: {metrics['val_acc_mean']:.2f}% ± {metrics['val_acc_std']:.2f}%")

# ============================================================
# CREATE SUMMARY DATAFRAME
# ============================================================

print("\n" + "=" * 60)
print("CREATING SUMMARY TABLE")
print("=" * 60)

summary_data = []

for data_type in DATA_TYPES:
    for aug_name in AUGMENTATIONS:
        for model_name in MODELS:
            if model_name in all_metrics[data_type][aug_name]:
                metrics = all_metrics[data_type][aug_name][model_name]

                summary_data.append({
                    'Model': model_name,
                    'Data Type': data_type,
                    'Augmentation': aug_name,
                    'Val Accuracy (%)': f"{metrics['val_acc_mean']:.2f} ± {metrics['val_acc_std']:.2f}",
                    'Val Loss': f"{metrics['val_loss_mean']:.4f} ± {metrics['val_loss_std']:.4f}",
                    'Train Accuracy (%)': f"{metrics['train_acc_mean']:.2f} ± {metrics['train_acc_std']:.2f}",
                    'Avg Epochs': f"{metrics['avg_epochs']:.0f}",
                    'Acc Mean': metrics['val_acc_mean'],  # For sorting
                })

# Create DataFrame
df_summary = pd.DataFrame(summary_data)

# Sort by validation accuracy
df_summary_sorted = df_summary.sort_values('Acc Mean', ascending=False)

# Display top 10
print("\n📊 TOP 10 CONFIGURATIONS:")
print("=" * 60)
display_cols = ['Model', 'Data Type', 'Augmentation', 'Val Accuracy (%)', 'Train Accuracy (%)']
print(df_summary_sorted[display_cols].head(10).to_string(index=False))

# Save full table
csv_path = os.path.join(TABLES_DIR, 'all_results_summary.csv')
df_summary_sorted.to_csv(csv_path, index=False)
print(f"\n✓ Full summary saved to: {csv_path}")

# ============================================================
# FIND BEST MODEL
# ============================================================

best_config = df_summary_sorted.iloc[0]

print("\n" + "=" * 60)
print("🏆 BEST MODEL CONFIGURATION")
print("=" * 60)
print(f"Model: {best_config['Model']}")
print(f"Data Type: {best_config['Data Type']}")
print(f"Augmentation: {best_config['Augmentation']}")
print(f"Validation Accuracy: {best_config['Val Accuracy (%)']}")
print(f"Training Accuracy: {best_config['Train Accuracy (%)']}")
print(f"Average Epochs: {best_config['Avg Epochs']}")

# Save best config
best_config_dict = {
    'model': best_config['Model'],
    'data_type': best_config['Data Type'],
    'augmentation': best_config['Augmentation'],
    'val_accuracy': best_config['Acc Mean'],
    'val_accuracy_formatted': best_config['Val Accuracy (%)']
}

best_config_path = os.path.join(RESULTS_DIR, 'best_model_config.json')
with open(best_config_path, 'w') as f:
    json.dump(best_config_dict, f, indent=2)

print(f"\n✓ Best config saved to: {best_config_path}")

print("\n" + "=" * 60)
print("STEP 6 COMPLETE!")
print("=" * 60)
print(f"\n📁 All results in: {RESULTS_DIR}")
print(f"   - Summary table: {TABLES_DIR}/all_results_summary.csv")
print(f"   - Best config: best_model_config.json")
print("\n✅ Ready for Step 7: Visualization Generation")

STEP 6: RESULTS ANALYSIS & METRICS CALCULATION

Output directories created:
  Figures: /content/drive/MyDrive/rabies_results/figures
  Tables: /content/drive/MyDrive/rabies_results/tables

LOADING TRAINING HISTORIES

✓ Loaded 72 training histories

CALCULATING METRICS
EfficientNet-B0      | raw      | TrivialAugmentWide   | Val Acc: 98.85% ± 1.63%
EfficientNet-B2      | raw      | TrivialAugmentWide   | Val Acc: 98.46% ± 0.54%
VGG16                | raw      | TrivialAugmentWide   | Val Acc: 98.47% ± 1.43%
ViT-B-16             | raw      | TrivialAugmentWide   | Val Acc: 92.31% ± 1.93%
EfficientNet-B0      | raw      | Geometric_Color      | Val Acc: 98.46% ± 1.43%
EfficientNet-B2      | raw      | Geometric_Color      | Val Acc: 99.62% ± 0.54%
VGG16                | raw      | Geometric_Color      | Val Acc: 99.23% ± 1.08%
ViT-B-16             | raw      | Geometric_Color      | Val Acc: 96.53% ± 2.49%
EfficientNet-B0      | raw      | Spatial_Blur         | Val Acc: 100.00% ± 0.00%
E

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

print("=" * 60)
print("STEP 7: VISUALIZATION GENERATION")
print("=" * 60)

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

# ============================================================
# FIGURE 1: LOSS CURVES (Training vs Validation)
# ============================================================

print("\n📈 Generating Figure 1: Loss Curves...")

fig, axes = plt.subplots(4, 3, figsize=(18, 20))
fig.suptitle('Training vs Validation Loss Curves (Overfitting Detection)',
             fontsize=16, fontweight='bold')

row_idx = 0
for model_name in MODELS:
    col_idx = 0
    for aug_name in AUGMENTATIONS:
        ax = axes[row_idx, col_idx]

        # Plot for both raw and cropped
        for data_type in DATA_TYPES:
            if model_name in histories[data_type][aug_name]:
                fold_histories = histories[data_type][aug_name][model_name]

                # Average loss across folds
                max_epochs = max(len(h['train_loss']) for h in fold_histories)
                train_losses = np.zeros(max_epochs)
                val_losses = np.zeros(max_epochs)
                counts = np.zeros(max_epochs)

                for h in fold_histories:
                    for i, (tl, vl) in enumerate(zip(h['train_loss'], h['val_loss'])):
                        train_losses[i] += tl
                        val_losses[i] += vl
                        counts[i] += 1

                train_losses = train_losses / np.maximum(counts, 1)
                val_losses = val_losses / np.maximum(counts, 1)

                epochs = range(1, len(train_losses) + 1)
                linestyle = '-' if data_type == 'raw' else '--'

                ax.plot(epochs, train_losses, linestyle=linestyle,
                       label=f'{data_type.capitalize()} Train', alpha=0.7)
                ax.plot(epochs, val_losses, linestyle=linestyle,
                       label=f'{data_type.capitalize()} Val', linewidth=2)

        ax.set_title(f'{model_name}\n{aug_name}', fontsize=9, fontweight='bold')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.3)

        col_idx += 1
    row_idx += 1

plt.tight_layout()
fig1_path = os.path.join(FIGURES_DIR, 'fig1_loss_curves.png')
plt.savefig(fig1_path, bbox_inches='tight')
plt.close()
print(f"✓ Saved: {fig1_path}")

# ============================================================
# FIGURE 2: ACCURACY CURVES
# ============================================================

print("\n📈 Generating Figure 2: Accuracy Curves...")

fig, axes = plt.subplots(4, 3, figsize=(18, 20))
fig.suptitle('Validation Accuracy Over Epochs', fontsize=16, fontweight='bold')

row_idx = 0
for model_name in MODELS:
    col_idx = 0
    for aug_name in AUGMENTATIONS:
        ax = axes[row_idx, col_idx]

        for data_type in DATA_TYPES:
            if model_name in histories[data_type][aug_name]:
                fold_histories = histories[data_type][aug_name][model_name]

                max_epochs = max(len(h['val_acc']) for h in fold_histories)
                val_accs = np.zeros(max_epochs)
                counts = np.zeros(max_epochs)

                for h in fold_histories:
                    for i, va in enumerate(h['val_acc']):
                        val_accs[i] += va
                        counts[i] += 1

                val_accs = val_accs / np.maximum(counts, 1)
                epochs = range(1, len(val_accs) + 1)

                linestyle = '-' if data_type == 'raw' else '--'
                ax.plot(epochs, val_accs, linestyle=linestyle,
                       label=data_type.capitalize(), linewidth=2)

        ax.set_title(f'{model_name}\n{aug_name}', fontsize=9, fontweight='bold')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy (%)')
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 105])

        col_idx += 1
    row_idx += 1

plt.tight_layout()
fig2_path = os.path.join(FIGURES_DIR, 'fig2_accuracy_curves.png')
plt.savefig(fig2_path, bbox_inches='tight')
plt.close()
print(f"✓ Saved: {fig2_path}")

# ============================================================
# FIGURE 3: BAR CHART COMPARISON (Raw vs Cropped)
# ============================================================

print("\n📊 Generating Figure 3: Bar Chart Comparison...")

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Model Performance: Raw vs Cropped Images',
             fontsize=14, fontweight='bold')

for idx, aug_name in enumerate(AUGMENTATIONS):
    ax = axes[idx]

    x_pos = np.arange(len(MODELS))
    width = 0.35

    raw_accs = []
    cropped_accs = []

    for model_name in MODELS:
        # Raw
        if model_name in all_metrics['raw'][aug_name]:
            raw_accs.append(all_metrics['raw'][aug_name][model_name]['val_acc_mean'])
        else:
            raw_accs.append(0)

        # Cropped
        if model_name in all_metrics['cropped'][aug_name]:
            cropped_accs.append(all_metrics['cropped'][aug_name][model_name]['val_acc_mean'])
        else:
            cropped_accs.append(0)

    bars1 = ax.bar(x_pos - width/2, raw_accs, width, label='Raw',
                   color='steelblue', alpha=0.8)
    bars2 = ax.bar(x_pos + width/2, cropped_accs, width, label='Cropped',
                   color='coral', alpha=0.8)

    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}%', ha='center', va='bottom', fontsize=8)

    ax.set_xlabel('Model')
    ax.set_ylabel('Validation Accuracy (%)')
    ax.set_title(aug_name, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(MODELS, rotation=45, ha='right')
    ax.legend()
    ax.set_ylim([0, 105])
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
fig3_path = os.path.join(FIGURES_DIR, 'fig3_raw_vs_cropped_comparison.png')
plt.savefig(fig3_path, bbox_inches='tight')
plt.close()
print(f"✓ Saved: {fig3_path}")

# ============================================================
# FIGURE 4: HEATMAP OF PERFORMANCE
# ============================================================

print("\n🔥 Generating Figure 4: Performance Heatmap...")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Performance Heatmap Across Configurations',
             fontsize=14, fontweight='bold')

for idx, data_type in enumerate(DATA_TYPES):
    ax = axes[idx]

    # Create matrix: rows = models, cols = augmentations
    heatmap_data = np.zeros((len(MODELS), len(AUGMENTATIONS)))

    for i, model_name in enumerate(MODELS):
        for j, aug_name in enumerate(AUGMENTATIONS):
            if model_name in all_metrics[data_type][aug_name]:
                heatmap_data[i, j] = all_metrics[data_type][aug_name][model_name]['val_acc_mean']
            else:
                heatmap_data[i, j] = np.nan

    sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn',
                xticklabels=AUGMENTATIONS, yticklabels=MODELS,
                vmin=70, vmax=100, ax=ax, cbar_kws={'label': 'Accuracy (%)'})

    ax.set_title(f'{data_type.capitalize()} Images', fontweight='bold')
    ax.set_xlabel('Augmentation Strategy')
    ax.set_ylabel('Model Architecture')

plt.tight_layout()
fig4_path = os.path.join(FIGURES_DIR, 'fig4_performance_heatmap.png')
plt.savefig(fig4_path, bbox_inches='tight')
plt.close()
print(f"✓ Saved: {fig4_path}")

# ============================================================
# FIGURE 5: OVERFITTING ANALYSIS
# ============================================================

print("\n⚠️  Generating Figure 5: Overfitting Analysis...")

overfitting_data = []

for data_type in DATA_TYPES:
    for aug_name in AUGMENTATIONS:
        for model_name in MODELS:
            if model_name in all_metrics[data_type][aug_name]:
                metrics = all_metrics[data_type][aug_name][model_name]
                gap = metrics['train_acc_mean'] - metrics['val_acc_mean']

                overfitting_data.append({
                    'Model': model_name,
                    'Aug': aug_name[:10],  # Shortened
                    'Data': data_type,
                    'Gap': gap,
                    'Config': f"{model_name}\n{aug_name[:10]}\n{data_type}"
                })

df_overfit = pd.DataFrame(overfitting_data)
df_overfit = df_overfit.sort_values('Gap', ascending=False)

fig, ax = plt.subplots(figsize=(14, 8))
colors = ['red' if g > 5 else 'orange' if g > 2 else 'green'
          for g in df_overfit['Gap']]

bars = ax.barh(range(len(df_overfit)), df_overfit['Gap'], color=colors, alpha=0.7)
ax.set_yticks(range(len(df_overfit)))
ax.set_yticklabels(df_overfit['Config'], fontsize=7)
ax.set_xlabel('Train-Val Accuracy Gap (%)', fontweight='bold')
ax.set_title('Overfitting Analysis: Train-Validation Accuracy Gap\n(Red>5%, Orange 2-5%, Green<2%)',
             fontweight='bold', fontsize=12)
ax.axvline(x=2, color='orange', linestyle='--', alpha=0.5, label='2% threshold')
ax.axvline(x=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
fig5_path = os.path.join(FIGURES_DIR, 'fig5_overfitting_analysis.png')
plt.savefig(fig5_path, bbox_inches='tight')
plt.close()
print(f"✓ Saved: {fig5_path}")

print("\n" + "=" * 60)
print("STEP 7 COMPLETE!")
print("=" * 60)
print(f"\n📊 All figures saved to: {FIGURES_DIR}")
print(f"   - fig1_loss_curves.png")
print(f"   - fig2_accuracy_curves.png")
print(f"   - fig3_raw_vs_cropped_comparison.png")
print(f"   - fig4_performance_heatmap.png")
print(f"   - fig5_overfitting_analysis.png")
print("\n✅ Ready for Step 8: Detailed Tables Generation")

STEP 7: VISUALIZATION GENERATION

📈 Generating Figure 1: Loss Curves...
✓ Saved: /content/drive/MyDrive/rabies_results/figures/fig1_loss_curves.png

📈 Generating Figure 2: Accuracy Curves...
✓ Saved: /content/drive/MyDrive/rabies_results/figures/fig2_accuracy_curves.png

📊 Generating Figure 3: Bar Chart Comparison...
✓ Saved: /content/drive/MyDrive/rabies_results/figures/fig3_raw_vs_cropped_comparison.png

🔥 Generating Figure 4: Performance Heatmap...
✓ Saved: /content/drive/MyDrive/rabies_results/figures/fig4_performance_heatmap.png

⚠️  Generating Figure 5: Overfitting Analysis...
✓ Saved: /content/drive/MyDrive/rabies_results/figures/fig5_overfitting_analysis.png

STEP 7 COMPLETE!

📊 All figures saved to: /content/drive/MyDrive/rabies_results/figures
   - fig1_loss_curves.png
   - fig2_accuracy_curves.png
   - fig3_raw_vs_cropped_comparison.png
   - fig4_performance_heatmap.png
   - fig5_overfitting_analysis.png

✅ Ready for Step 8: Detailed Tables Generation


In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import json

print("=" * 60)
print("STEP 8: RETRAIN BEST MODEL")
print("=" * 60)

# Best configuration
BEST_MODEL = 'EfficientNet-B0'
BEST_DATA_TYPE = 'cropped'
BEST_AUGMENTATION = 'Geometric_Color'

print(f"\n🏆 Best Model Configuration:")
print(f"   Model: {BEST_MODEL}")
print(f"   Data Type: {BEST_DATA_TYPE}")
print(f"   Augmentation: {BEST_AUGMENTATION}")

# Paths
BASE_CROPPED = "/content/drive/MyDrive/rabies_cropped_split"
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"
BEST_MODEL_PATH = os.path.join(RESULTS_DIR, "BEST_MODEL_EfficientNetB0.pth")

# Check if model already exists
if os.path.exists(BEST_MODEL_PATH):
    print(f"\n✓ Best model checkpoint already exists!")
    print(f"   Path: {BEST_MODEL_PATH}")
    print(f"\n   Skipping retraining. Using existing checkpoint.")
    model_exists = True
else:
    print(f"\n⚠️  Best model checkpoint not found. Retraining...")
    model_exists = False

# ============================================================
# RETRAIN IF NOT EXISTS
# ============================================================

if not model_exists:
    print("\n" + "=" * 60)
    print("RETRAINING BEST MODEL ON FULL TRAINING DATA")
    print("=" * 60)

    # Load full training dataset (all folds combined)
    train_dir = os.path.join(BASE_CROPPED, f'train_{BEST_AUGMENTATION}')
    full_train_dataset = RabiesDataset(train_dir, transform=train_transform)

    print(f"\nTraining dataset loaded:")
    print(f"   Total samples: {len(full_train_dataset)}")
    print(f"   Negatif: {full_train_dataset.labels.count(0)}")
    print(f"   Positif: {full_train_dataset.labels.count(1)}")

    # Create dataloader
    train_loader = DataLoader(
        full_train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )

    # Initialize model
    print(f"\nInitializing {BEST_MODEL}...")
    model = get_model(BEST_MODEL).to(DEVICE)

    # Setup training components
    optimizer, scheduler = setup_optimizer_scheduler(model)

    print(f"\nTraining configuration:")
    print(f"   Max epochs: {MAX_EPOCHS}")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Learning rate: {LEARNING_RATE}")
    print(f"   Device: {DEVICE}")

    # Training loop
    print("\n" + "=" * 60)
    print("TRAINING IN PROGRESS...")
    print("=" * 60)

    best_train_acc = 0
    training_history = {
        'train_loss': [],
        'train_acc': []
    }

    for epoch in range(MAX_EPOCHS):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / total
        epoch_acc = 100. * correct / total

        training_history['train_loss'].append(epoch_loss)
        training_history['train_acc'].append(epoch_acc)

        if epoch_acc > best_train_acc:
            best_train_acc = epoch_acc

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{MAX_EPOCHS}] - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%")

    print(f"\n✓ Training complete!")
    print(f"   Best training accuracy: {best_train_acc:.2f}%")

    # Save model
    torch.save(model.state_dict(), BEST_MODEL_PATH)
    print(f"\n✓ Model saved to: {BEST_MODEL_PATH}")

    # Save training history
    history_path = os.path.join(RESULTS_DIR, "best_model_training_history.json")
    with open(history_path, 'w') as f:
        json.dump(training_history, f, indent=2)
    print(f"✓ Training history saved to: {history_path}")

    # Clean up
    del model, optimizer
    torch.cuda.empty_cache()

print("\n" + "=" * 60)
print("STEP 8 COMPLETE!")
print("=" * 60)
print(f"\n✅ Best model checkpoint ready at: {BEST_MODEL_PATH}")
print(f"\n📌 Next: Step 9 - Test Set Evaluation")

STEP 8: RETRAIN BEST MODEL

🏆 Best Model Configuration:
   Model: EfficientNet-B0
   Data Type: cropped
   Augmentation: Geometric_Color

⚠️  Best model checkpoint not found. Retraining...

RETRAINING BEST MODEL ON FULL TRAINING DATA

Training dataset loaded:
   Total samples: 432
   Negatif: 88
   Positif: 344

Initializing EfficientNet-B0...
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 128MB/s]



Training configuration:
   Max epochs: 25
   Batch size: 32
   Learning rate: 0.0001
   Device: cuda

TRAINING IN PROGRESS...
Epoch [5/25] - Loss: 0.0568, Acc: 98.84%
Epoch [10/25] - Loss: 0.0185, Acc: 99.77%
Epoch [15/25] - Loss: 0.0114, Acc: 99.54%
Epoch [20/25] - Loss: 0.0024, Acc: 100.00%
Epoch [25/25] - Loss: 0.0011, Acc: 100.00%

✓ Training complete!
   Best training accuracy: 100.00%

✓ Model saved to: /content/drive/MyDrive/rabies_results/BEST_MODEL_EfficientNetB0.pth
✓ Training history saved to: /content/drive/MyDrive/rabies_results/best_model_training_history.json

STEP 8 COMPLETE!

✅ Best model checkpoint ready at: /content/drive/MyDrive/rabies_results/BEST_MODEL_EfficientNetB0.pth

📌 Next: Step 9 - Test Set Evaluation


In [11]:
import torch
import numpy as np
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, fbeta_score, confusion_matrix,
                             classification_report)
import matplotlib.pyplot as plt
import seaborn as sns
import json

print("=" * 60)
print("STEP 9: TEST SET EVALUATION")
print("=" * 60)

# Configuration
BEST_MODEL = 'EfficientNet-B0'
BEST_DATA_TYPE = 'cropped'
BASE_CROPPED = "/content/drive/MyDrive/rabies_cropped_split"
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"
BEST_MODEL_PATH = os.path.join(RESULTS_DIR, "BEST_MODEL_EfficientNetB0.pth")

# ============================================================
# LOAD TEST DATA
# ============================================================

print("\n📂 Loading test dataset...")

test_dir = os.path.join(BASE_CROPPED, 'test')
test_dataset = RabiesDataset(test_dir, transform=val_transform)

print(f"Test dataset loaded:")
print(f"   Total samples: {len(test_dataset)}")
print(f"   Negatif: {test_dataset.labels.count(0)}")
print(f"   Positif: {test_dataset.labels.count(1)}")

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# ============================================================
# LOAD BEST MODEL
# ============================================================

print(f"\n🔄 Loading best model from checkpoint...")

model = get_model(BEST_MODEL).to(DEVICE)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

print(f"✓ Model loaded: {BEST_MODEL}")

# ============================================================
# EVALUATE ON TEST SET
# ============================================================

print("\n" + "=" * 60)
print("EVALUATING ON TEST SET (NEVER SEEN BEFORE)")
print("=" * 60)

all_predictions = []
all_labels = []
all_probs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Convert to numpy arrays
y_true = np.array(all_labels)
y_pred = np.array(all_predictions)
y_probs = np.array(all_probs)

# ============================================================
# CALCULATE METRICS
# ============================================================

print("\n📊 Calculating metrics...")

# Calculate all metrics
test_accuracy = accuracy_score(y_true, y_pred) * 100
test_precision = precision_score(y_true, y_pred, average='weighted') * 100
test_recall = recall_score(y_true, y_pred, average='weighted') * 100
test_f1 = f1_score(y_true, y_pred, average='weighted') * 100
test_f2 = fbeta_score(y_true, y_pred, beta=2, average='weighted') * 100

# Per-class metrics
precision_per_class = precision_score(y_true, y_pred, average=None)
recall_per_class = recall_score(y_true, y_pred, average=None)
f1_per_class = f1_score(y_true, y_pred, average=None)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# ============================================================
# DISPLAY RESULTS
# ============================================================

print("\n" + "=" * 60)
print("🎯 FINAL TEST SET RESULTS (FOR RESEARCH PAPER)")
print("=" * 60)

print(f"\nOverall Metrics:")
print(f"   Accuracy:  {test_accuracy:.2f}%")
print(f"   Precision: {test_precision:.2f}%")
print(f"   Recall:    {test_recall:.2f}%")
print(f"   F1-Score:  {test_f1:.2f}%")
print(f"   F2-Score:  {test_f2:.2f}%")

print(f"\nPer-Class Metrics:")
print(f"   Negatif (Class 0):")
print(f"      Precision: {precision_per_class[0]*100:.2f}%")
print(f"      Recall:    {recall_per_class[0]*100:.2f}%")
print(f"      F1-Score:  {f1_per_class[0]*100:.2f}%")

print(f"\n   Positif (Class 1):")
print(f"      Precision: {precision_per_class[1]*100:.2f}%")
print(f"      Recall:    {recall_per_class[1]*100:.2f}%")
print(f"      F1-Score:  {f1_per_class[1]*100:.2f}%")

print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"                Negatif  Positif")
print(f"   Actual Negatif    {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"          Positif    {cm[1,0]:3d}      {cm[1,1]:3d}")

# ============================================================
# SAVE METRICS
# ============================================================

test_results = {
    'model': BEST_MODEL,
    'data_type': BEST_DATA_TYPE,
    'test_set_size': len(test_dataset),
    'metrics': {
        'accuracy': float(test_accuracy),
        'precision': float(test_precision),
        'recall': float(test_recall),
        'f1_score': float(test_f1),
        'f2_score': float(test_f2)
    },
    'per_class_metrics': {
        'negatif': {
            'precision': float(precision_per_class[0] * 100),
            'recall': float(recall_per_class[0] * 100),
            'f1_score': float(f1_per_class[0] * 100)
        },
        'positif': {
            'precision': float(precision_per_class[1] * 100),
            'recall': float(recall_per_class[1] * 100),
            'f1_score': float(f1_per_class[1] * 100)
        }
    },
    'confusion_matrix': cm.tolist()
}

results_path = os.path.join(RESULTS_DIR, 'test_set_results.json')
with open(results_path, 'w') as f:
    json.dump(test_results, f, indent=2)

print(f"\n✓ Test results saved to: {results_path}")

# ============================================================
# PLOT CONFUSION MATRIX
# ============================================================

print("\n📊 Generating confusion matrix visualization...")

fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Negatif', 'Positif'],
            yticklabels=['Negatif', 'Positif'],
            ax=ax)

ax.set_xlabel('Predicted Label', fontweight='bold', fontsize=12)
ax.set_ylabel('True Label', fontweight='bold', fontsize=12)
ax.set_title(f'Confusion Matrix - Test Set\n{BEST_MODEL} ({BEST_DATA_TYPE.capitalize()} Images)\nAccuracy: {test_accuracy:.2f}%',
            fontweight='bold', fontsize=14)

# Add percentage annotations
for i in range(2):
    for j in range(2):
        percentage = cm[i, j] / cm[i].sum() * 100
        ax.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)',
               ha='center', va='center', fontsize=10, color='gray')

plt.tight_layout()
cm_path = os.path.join(FIGURES_DIR, 'test_set_confusion_matrix.png')
plt.savefig(cm_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Confusion matrix saved to: {cm_path}")

# ============================================================
# DETAILED CLASSIFICATION REPORT
# ============================================================

print("\n📄 Generating detailed classification report...")

report = classification_report(y_true, y_pred,
                               target_names=['Negatif', 'Positif'],
                               digits=4)

print("\n" + "=" * 60)
print("CLASSIFICATION REPORT")
print("=" * 60)
print(report)

report_path = os.path.join(RESULTS_DIR, 'test_set_classification_report.txt')
with open(report_path, 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("TEST SET EVALUATION - CLASSIFICATION REPORT\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Model: {BEST_MODEL}\n")
    f.write(f"Data Type: {BEST_DATA_TYPE}\n")
    f.write(f"Test Set Size: {len(test_dataset)}\n\n")
    f.write(report)
    f.write("\n\nConfusion Matrix:\n")
    f.write(f"              Predicted\n")
    f.write(f"              Negatif  Positif\n")
    f.write(f"Actual Negatif    {cm[0,0]:3d}      {cm[0,1]:3d}\n")
    f.write(f"       Positif    {cm[1,0]:3d}      {cm[1,1]:3d}\n")

print(f"\n✓ Classification report saved to: {report_path}")

print("\n" + "=" * 60)
print("STEP 9 COMPLETE!")
print("=" * 60)
print(f"\n✅ Test set evaluation complete!")
print(f"\n📌 Next: Step 10 - Grad-CAM Generation")

STEP 9: TEST SET EVALUATION

📂 Loading test dataset...
Test dataset loaded:
   Total samples: 24
   Negatif: 5
   Positif: 19

🔄 Loading best model from checkpoint...
✓ Model loaded: EfficientNet-B0

EVALUATING ON TEST SET (NEVER SEEN BEFORE)

📊 Calculating metrics...

🎯 FINAL TEST SET RESULTS (FOR RESEARCH PAPER)

Overall Metrics:
   Accuracy:  100.00%
   Precision: 100.00%
   Recall:    100.00%
   F1-Score:  100.00%
   F2-Score:  100.00%

Per-Class Metrics:
   Negatif (Class 0):
      Precision: 100.00%
      Recall:    100.00%
      F1-Score:  100.00%

   Positif (Class 1):
      Precision: 100.00%
      Recall:    100.00%
      F1-Score:  100.00%

Confusion Matrix:
                Predicted
                Negatif  Positif
   Actual Negatif      5        0
          Positif      0       19

✓ Test results saved to: /content/drive/MyDrive/rabies_results/test_set_results.json

📊 Generating confusion matrix visualization...
✓ Confusion matrix saved to: /content/drive/MyDrive/rabies_re

In [13]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os

print("=" * 60)
print("STEP 10: GRAD-CAM GENERATION")
print("=" * 60)

# Configuration
BEST_MODEL = 'EfficientNet-B0'
BEST_DATA_TYPE = 'cropped'
BASE_CROPPED = "/content/drive/MyDrive/rabies_cropped_split"
RESULTS_DIR = "/content/drive/MyDrive/rabies_results"
GRADCAM_DIR = os.path.join(RESULTS_DIR, "gradcam")
BEST_MODEL_PATH = os.path.join(RESULTS_DIR, "BEST_MODEL_EfficientNetB0.pth")

os.makedirs(GRADCAM_DIR, exist_ok=True)

# ============================================================
# GRAD-CAM CLASS
# ============================================================

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        # Register hooks
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def generate_cam(self, input_image, target_class=None):
        # Forward pass
        model_output = self.model(input_image)

        if target_class is None:
            target_class = model_output.argmax(dim=1)

        # Backward pass
        self.model.zero_grad()
        class_loss = model_output[0, target_class]
        class_loss.backward()

        # Get weights
        gradients = self.gradients[0]
        activations = self.activations[0]

        # Global average pooling
        weights = gradients.mean(dim=(1, 2))

        # Weighted combination (keep on same device)
        cam = torch.zeros(activations.shape[1:], dtype=torch.float32, device=activations.device)
        for i, w in enumerate(weights):
            cam += w * activations[i]

        # ReLU
        cam = F.relu(cam)

        # Normalize
        cam = cam - cam.min()
        if cam.max() > 0:
            cam = cam / cam.max()

        return cam.cpu().numpy()

# ============================================================
# LOAD MODEL
# ============================================================

print("\n🔄 Loading best model...")

model = get_model(BEST_MODEL).to(DEVICE)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

# Get target layer (last convolutional layer)
# For EfficientNet: features[-1]
target_layer = model.features[-1]

print(f"✓ Model loaded: {BEST_MODEL}")
print(f"✓ Target layer for Grad-CAM: {target_layer.__class__.__name__}")

# Initialize Grad-CAM
gradcam = GradCAM(model, target_layer)

# ============================================================
# GENERATE GRAD-CAM FOR SAMPLE IMAGES
# ============================================================

print("\n" + "=" * 60)
print("GENERATING GRAD-CAM VISUALIZATIONS")
print("=" * 60)

# Load test set
test_dir = os.path.join(BASE_CROPPED, 'test')
test_dataset = RabiesDataset(test_dir, transform=val_transform)

# Select samples (2 from each class)
negatif_indices = [i for i, label in enumerate(test_dataset.labels) if label == 0]
positif_indices = [i for i, label in enumerate(test_dataset.labels) if label == 1]

# Take first 2 from each class
sample_indices = negatif_indices[:2] + positif_indices[:2]

print(f"\nGenerating Grad-CAM for {len(sample_indices)} sample images...")

def apply_colormap_on_image(org_im, activation_map):
    """Apply colormap on activation map"""
    heatmap = cv2.applyColorMap(np.uint8(255 * activation_map), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    # Resize to match original image
    org_im_resized = cv2.resize(np.array(org_im), (224, 224))
    org_im_resized = np.float32(org_im_resized) / 255

    # Superimpose
    cam_image = heatmap * 0.4 + org_im_resized * 0.6
    cam_image = cam_image / cam_image.max()

    return np.uint8(255 * cam_image)

# Generate Grad-CAMs
for idx, sample_idx in enumerate(sample_indices):
    # Load original image
    img_path = test_dataset.samples[sample_idx]
    original_image = Image.open(img_path).convert('RGB')
    true_label = test_dataset.labels[sample_idx]

    # Prepare input
    input_tensor = val_transform(original_image).unsqueeze(0).to(DEVICE)

    # Get prediction
    with torch.no_grad():
        output = model(input_tensor)
        pred_prob = F.softmax(output, dim=1)
        pred_class = output.argmax(dim=1).item()
        confidence = pred_prob[0, pred_class].item() * 100

    # Generate Grad-CAM
    cam = gradcam.generate_cam(input_tensor, target_class=pred_class)

    # Resize CAM to match input size
    cam_resized = cv2.resize(cam, (224, 224))

    # Create visualization
    cam_image = apply_colormap_on_image(original_image, cam_resized)

    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Original image
    axes[0].imshow(original_image)
    axes[0].set_title('Original Image', fontweight='bold')
    axes[0].axis('off')

    # Heatmap
    axes[1].imshow(cam_resized, cmap='jet')
    axes[1].set_title('Grad-CAM Heatmap', fontweight='bold')
    axes[1].axis('off')

    # Overlay
    axes[2].imshow(cam_image)
    axes[2].set_title('Grad-CAM Overlay', fontweight='bold')
    axes[2].axis('off')

    # Add info
    true_label_name = 'Positif' if true_label == 1 else 'Negatif'
    pred_label_name = 'Positif' if pred_class == 1 else 'Negatif'

    fig.suptitle(f'Sample {idx+1}: True={true_label_name}, Predicted={pred_label_name} ({confidence:.1f}% confidence)',
                fontsize=14, fontweight='bold')

    plt.tight_layout()

    # Save
    save_path = os.path.join(GRADCAM_DIR, f'gradcam_sample_{idx+1}_{true_label_name.lower()}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✓ Generated Grad-CAM {idx+1}/{len(sample_indices)}: {true_label_name}")

# ============================================================
# CREATE COMBINED FIGURE
# ============================================================

print("\n📊 Creating combined Grad-CAM figure...")

fig, axes = plt.subplots(4, 3, figsize=(15, 20))
fig.suptitle(f'Grad-CAM Visualizations - {BEST_MODEL} on {BEST_DATA_TYPE.capitalize()} Images',
            fontsize=16, fontweight='bold')

for idx, sample_idx in enumerate(sample_indices):
    img_path = test_dataset.samples[sample_idx]
    original_image = Image.open(img_path).convert('RGB')
    true_label = test_dataset.labels[sample_idx]

    input_tensor = val_transform(original_image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(input_tensor)
        pred_class = output.argmax(dim=1).item()

    cam = gradcam.generate_cam(input_tensor, target_class=pred_class)
    cam_resized = cv2.resize(cam, (224, 224))
    cam_image = apply_colormap_on_image(original_image, cam_resized)

    # Plot original
    axes[idx, 0].imshow(original_image)
    axes[idx, 0].set_title('Original', fontweight='bold')
    axes[idx, 0].axis('off')

    # Plot heatmap
    axes[idx, 1].imshow(cam_resized, cmap='jet')
    axes[idx, 1].set_title('Heatmap', fontweight='bold')
    axes[idx, 1].axis('off')

    # Plot overlay
    axes[idx, 2].imshow(cam_image)
    true_label_name = 'Positif' if true_label == 1 else 'Negatif'
    axes[idx, 2].set_title(f'Overlay ({true_label_name})', fontweight='bold')
    axes[idx, 2].axis('off')

plt.tight_layout()
combined_path = os.path.join(GRADCAM_DIR, 'gradcam_combined.png')
plt.savefig(combined_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Combined figure saved: {combined_path}")

print("\n" + "=" * 60)
print("STEP 10 COMPLETE!")
print("=" * 60)
print(f"\n✅ Grad-CAM visualizations generated!")
print(f"\n📁 All Grad-CAM images saved to: {GRADCAM_DIR}")
print(f"   - Individual samples: gradcam_sample_X.png")
print(f"   - Combined figure: gradcam_combined.png")
print("\n🎉 ALL ANALYSIS COMPLETE! Everything is saved and ready for your paper!")

STEP 10: GRAD-CAM GENERATION

🔄 Loading best model...
✓ Model loaded: EfficientNet-B0
✓ Target layer for Grad-CAM: Conv2dNormActivation

GENERATING GRAD-CAM VISUALIZATIONS

Generating Grad-CAM for 4 sample images...
✓ Generated Grad-CAM 1/4: Negatif
✓ Generated Grad-CAM 2/4: Negatif
✓ Generated Grad-CAM 3/4: Positif
✓ Generated Grad-CAM 4/4: Positif

📊 Creating combined Grad-CAM figure...
✓ Combined figure saved: /content/drive/MyDrive/rabies_results/gradcam/gradcam_combined.png

STEP 10 COMPLETE!

✅ Grad-CAM visualizations generated!

📁 All Grad-CAM images saved to: /content/drive/MyDrive/rabies_results/gradcam
   - Individual samples: gradcam_sample_X.png
   - Combined figure: gradcam_combined.png

🎉 ALL ANALYSIS COMPLETE! Everything is saved and ready for your paper!
